In [1]:
import pickle
import numpy as np
import seaborn as sns
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, concatenate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model
from keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import statistics

In [2]:
path = "/kaggle/input/dataset-wrist-filt/combined_wrist_filtered.pkl"

In [3]:
df_wrist = pd.read_pickle(path)
df_wrist.head(10)

,id,ACC_x,ACC_y,ACC_z,BVP,EDA,TEMP,label
214583,2.0,39.469502,27.912214,29.039982,-109.222518,1.637703,35.809887,1.0
214584,2.0,38.595181,27.796006,29.093324,-108.599068,1.637493,35.809884,1.0
214585,2.0,37.719606,27.683257,29.114913,-107.981064,1.637285,35.809882,1.0
214586,2.0,36.849917,27.575278,29.103982,-107.368689,1.637080,35.809879,1.0
214587,2.0,35.993018,27.473250,29.060322,-106.762018,1.636878,35.809876,1.0
214588,2.0,35.155509,27.378208,28.984297,-106.161005,1.636678,35.809873,1.0
214589,2.0,34.343617,27.291020,28.876853,-105.565485,1.636481,35.809871,1.0
214590,2.0,33.563132,27.212380,28.739512,-104.975172,1.636287,35.809868,1.0
214591,2.0,32.819356,27.142793,28.574360,-104.389668,1.636095,35.809866,1.0
214592,2.0,32.117049,27.082575,28.384019,-103.808476,1.635906,35.809863,1.0


In [4]:
df_wrist.describe()

,id,ACC_x,ACC_y,ACC_z,BVP,EDA,TEMP,label
count,2.320640e+07,2.320640e+07,2.320640e+07,2.320640e+07,2.320640e+07,2.320640e+07,2.320640e+07,2.320640e+07
mean,9.399494e+00,1.661687e+01,-5.119978e+00,1.044325e+01,9.835437e-03,1.962381e+00,3.300757e+01,1.636945e+00
std,4.705348e+00,4.273679e+01,2.984401e+01,3.057830e+01,6.458049e+01,2.650179e+00,1.470426e+00,7.533762e-01
min,2.000000e+00,-1.431664e+02,-1.419849e+02,-1.489489e+02,-1.618099e+03,7.319591e-02,2.932185e+01,1.000000e+00
25%,5.000000e+00,-2.799696e+01,-2.524123e+01,-6.998641e+00,-1.552436e+01,3.084203e-01,3.226904e+01,1.000000e+00
50%,9.000000e+00,3.456062e+01,-6.956822e+00,1.198514e+01,2.161803e+00,8.598483e-01,3.315138e+01,1.000000e+00
75%,1.400000e+01,5.400000e+01,1.095193e+01,3.404234e+01,1.716484e+01,2.652989e+00,3.400144e+01,2.000000e+00
max,1.700000e+01,1.767483e+02,1.362818e+02,1.596128e+02,1.789086e+03,1.594576e+01,3.597752e+01,3.000000e+00


In [5]:
df_wrist.shape

(23206404, 8)

In [6]:
df_wrist.groupby(['id', 'label']).count()

ACC_x   ACC_y   ACC_z     BVP     EDA    TEMP
id   label                                                
2.0  1.0    800800  800800  800800  800800  800800  800800
     2.0    430500  430500  430500  430500  430500  430500
     3.0    253400  253400  253400  253400  253400  253400
3.0  1.0    798000  798000  798000  798000  798000  798000
     2.0    448000  448000  448000  448000  448000  448000
     3.0    262500  262500  262500  262500  262500  262500
4.0  1.0    810601  810601  810601  810601  810601  810601
     2.0    444500  444500  444500  444500  444500  444500
     3.0    260400  260400  260400  260400  260400  260400
5.0  1.0    838600  838600  838600  838600  838600  838600
     2.0    451500  451500  451500  451500  451500  451500
     3.0    261800  261800  261800  261800  261800  261800
6.0  1.0    826000  826000  826000  826000  826000  826000
     2.0    455000  455000  455000  455000  455000  455000
     3.0    260400  260400  260400  260400  260400  260400
7.0  1.0    830200  830200  830200  830200  830200  830200
     2.0    448000  448000  448000  448000  448000  448000
     3.0    260401  260401  260401  260401  260401  260401
8.0  1.0    818300  818300  818300  818300  818300  818300
     2.0    469000  469000  469000  469000  469000  469000
     3.0    258999  258999  258999  258999  258999  258999
9.0  1.0    826000  826000  826000  826000  826000  826000
     2.0    451500  451500  451500  451500  451500  451500
     3.0    260400  260400  260400  260400  260400  260400
10.0 1.0    826000  826000  826000  826000  826000  826000
     2.0    507500  507500  507500  507500  507500  507500
     3.0    260400  260400  260400  260400  260400  260400
11.0 1.0    826000  826000  826000  826000  826000  826000
     2.0    476000  476000  476000  476000  476000  476000
     3.0    257600  257600  257600  257600  257600  257600
13.0 1.0    826001  826001  826001  826001  826001  826001
     2.0    464800  464800  464800  464800  464800  464800
     3.0    267400  267400  267400  267400  267400  267400
14.0 1.0    826000  826000  826000  826000  826000  826000
     2.0    472500  472500  472500  472500  472500  472500
     3.0    260401  260401  260401  260401  260401  260401
15.0 1.0    822500  822500  822500  822500  822500  822500
     2.0    480200  480200  480200  480200  480200  480200
     3.0    260400  260400  260400  260400  260400  260400
16.0 1.0    826000  826000  826000  826000  826000  826000
     2.0    471101  471101  471101  471101  471101  471101
     3.0    257600  257600  257600  257600  257600  257600
17.0 1.0    826700  826700  826700  826700  826700  826700
     2.0    506100  506100  506100  506100  506100  506100
     3.0    260400  260400  260400  260400  260400  260400

In [7]:
# Crear un modelo CNN para clasificación binaria para los datos recopilados de muñeca
sampling_rate = 700  # Hz
window_time = 5  # segundos
sequence_length = sampling_rate * window_time
step_size = sequence_length // 2

# Extraer features y labels
X = df_wrist[['ACC_x', 'ACC_y', 'ACC_z', 'BVP', 'EDA', 'TEMP']].values
y = df_wrist['label'].values

# Crear segmentos con overlapping windows
segments = []
labels = []
for i in range(0, len(X) - sequence_length, step_size):
    segment = X[i:i + sequence_length]
    label = y[i + sequence_length - 1]  
    segments.append(segment)
    labels.append(label)

# Convertir a vector 
X_segments = np.array(segments)
y_segments = np.array(labels)
# One hot encoding, clasificar label 1(fase neutral) y 3(diversión) a un grupo(no estrés) y label 2(estrés) a otro
y_segments_binary = (y_segments != 2).astype(int)

# Redimensionar X_segments para adaptar a la dimensión de entrada para LSTM
num_features = X_segments.shape[2]
X_segments_reshaped = X_segments.reshape((X_segments.shape[0], X_segments.shape[1], num_features))

# Definir el input layer para todos los sensores
input_layer = Input(shape=(sequence_length, num_features))

# Función para crear un bloque convolucional de 1D 
def conv_block(x, filters, kernel_size, stride, pool_size, pool_stride):
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=stride, activation='relu')(x)
    x = MaxPooling1D(pool_size=pool_size, strides=pool_stride)(x)
    return x

# Aplicar el bloque convolucional al input
output = conv_block(input_layer, filters=8, kernel_size=15, stride=2, pool_size=4, pool_stride=4)

# Conexión completa entre layers
x = Dense(32, activation='relu')(Flatten()(output))
x = Dense(16, activation='relu')(x)

# Output layer
output_layer = Dense(1, activation='sigmoid')(x)

# Generar el modelo
model_w_binary = Model(inputs=input_layer, outputs=output_layer)

# Imprimir por pantalla el resumen del modelo
model_w_binary.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3500, 6)]         0         
                                                                 
 conv1d (Conv1D)             (None, 1743, 8)           728       
                                                                 
 max_pooling1d (MaxPooling1  (None, 435, 8)            0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 3480)              0         
                                                                 
 dense (Dense)               (None, 32)                111392    
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                             

In [8]:
# Normalización de datos
scaler = StandardScaler()
X_segments_normalized = scaler.fit_transform(X_segments_reshaped.reshape(-1, num_features)).reshape(X_segments_reshaped.shape)

# Dividir los datos para entrenamiento(80%) y para prueba(20%)
X_train, X_test, y_train, y_test = train_test_split(X_segments_normalized, y_segments_binary, test_size=0.2, random_state=42)

# Ejecutar el modelo
from keras.optimizers import Adam
optimizer = Adam(lr=0.01)
model_w_binary.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model_w_binary.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluar la precisión con los datos de prueba
loss, accuracy = model_w_binary.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación binaria para datos de muñeca: {accuracy * 100:.2f}%')

Epoch 1/20
332/332 [==============================] - 8s 20ms/step - loss: 0.2505 - accuracy: 0.8925 - val_loss: 0.1823 - val_accuracy: 0.9261
Epoch 2/20
332/332 [==============================] - 6s 18ms/step - loss: 0.1227 - accuracy: 0.9554 - val_loss: 0.0876 - val_accuracy: 0.9702
Epoch 3/20
332/332 [==============================] - 6s 18ms/step - loss: 0.0733 - accuracy: 0.9767 - val_loss: 0.0937 - val_accuracy: 0.9729
Epoch 4/20
332/332 [==============================] - 6s 18ms/step - loss: 0.0638 - accuracy: 0.9793 - val_loss: 0.0810 - val_accuracy: 0.9774
Epoch 5/20
332/332 [==============================] - 6s 18ms/step - loss: 0.0563 - accuracy: 0.9816 - val_loss: 0.0976 - val_accuracy: 0.9698
Epoch 6/20
332/332 [==============================] - 6s 18ms/step - loss: 0.0528 - accuracy: 0.9841 - val_loss: 0.0506 - val_accuracy: 0.9838
Epoch 7/20
332/332 [==============================] - 6s 17ms/step - loss: 0.0430 - accuracy: 0.9851 - val_loss: 0.0608 - val_accuracy: 0.9857

In [9]:
y_pred_binary = (model_w_binary.predict(X_test) > 0.5).astype(int)
y_true_binary = y_test 
# Calcular F1-score, precisión y recall
f1 = f1_score(y_true_binary, y_pred_binary)
precision = precision_score(y_true_binary, y_pred_binary)
recall = recall_score(y_true_binary, y_pred_binary)

# Imprimir los reesultados
print("Resultados del modelo de clasificación binaria para datos de muñeca: ")
print(f'F1-score: {f1:.4f}')
print(f'Precisión: {precision:.4f}')
print(f'Recall: {recall:.4f}') # Recall es la Tasa de verdaderos positivos

# Imprimir la matriz de confusión
conf_matrix = confusion_matrix(y_true_binary, y_pred_binary)
print('Matriz de confusión:')
print(conf_matrix)

# Imprimir por pantalla el resumen del modelo
model_w_binary.summary()

83/83 [==============================] - 1s 6ms/step
Resultados del modelo de clasificación binaria para datos de muñeca: 
F1-score: 0.9893
Precisión: 0.9907
Recall: 0.9880
Matriz de confusión:
[[ 805   17]
 [  22 1808]]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3500, 6)]         0         
                                                                 
 conv1d (Conv1D)             (None, 1743, 8)           728       
                                                                 
 max_pooling1d (MaxPooling1  (None, 435, 8)            0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 3480)              0         
                                                                 
 dense (Dense)               (None, 32

In [10]:
from keras.layers import Dropout
# Crear un modelo CNN para clasificación binaria para los datos recopilados de muñeca
sampling_rate = 700  # Hz
window_time = 5  # segundos
sequence_length = sampling_rate * window_time
step_size = sequence_length // 2

# Extraer features y labels
X = df_wrist[['ACC_x', 'ACC_y', 'ACC_z', 'BVP', 'EDA', 'TEMP']].values
y = df_wrist['label'].values

# Crear segmentos con overlapping windows
segments = []
labels = []
for i in range(0, len(X) - sequence_length, step_size):
    segment = X[i:i + sequence_length]
    label = y[i + sequence_length - 1]  
    segments.append(segment)
    labels.append(label)

# Convertir a vector 
X_segments = np.array(segments)
y_segments = np.array(labels)
# One hot encoding, clasificar label 1(fase neutral) y 3(diversión) a un grupo(no estrés) y label 2(estrés) a otro
y_segments_binary = (y_segments != 2).astype(int)

# Redimensionar X_segments para adaptar a la dimensión de entrada para LSTM
num_features = X_segments.shape[2]
X_segments_reshaped = X_segments.reshape((X_segments.shape[0], X_segments.shape[1], num_features))

# Definir el input layer para todos los sensores
input_layer = Input(shape=(sequence_length, num_features))

def conv_block(x, filters, kernel_size, stride, pool_size, pool_stride):
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=stride, activation='relu')(x)
    x = MaxPooling1D(pool_size=pool_size, strides=pool_stride)(x)
    return x

output = conv_block(input_layer, filters=16, kernel_size=15, stride=2, pool_size=4, pool_stride=4)
output = conv_block(output, filters=32, kernel_size=10, stride=1, pool_size=2, pool_stride=2)

# Conexión completa entre layers
x = Dense(32, activation='relu')(Flatten()(output))
x = Dropout(0.5)(x)
x = Dense(16, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layer
output_layer = Dense(1, activation='sigmoid')(x)

# Generar el modelo
model_w_binary = Model(inputs=input_layer, outputs=output_layer)


In [11]:
# Normalización de datos
scaler = StandardScaler()
X_segments_normalized = scaler.fit_transform(X_segments_reshaped.reshape(-1, num_features)).reshape(X_segments_reshaped.shape)

# Dividir los datos para entrenamiento(80%) y para prueba(20%)
X_train, X_test, y_train, y_test = train_test_split(X_segments_normalized, y_segments_binary, test_size=0.2, random_state=42)

# Ejecutar el modelo
from keras.optimizers import Adam
optimizer = Adam(lr=0.01)
model_w_binary.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model_w_binary.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluar la precisión con los datos de prueba
loss, accuracy = model_w_binary.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación binaria para datos de muñeca: {accuracy * 100:.2f}%')

Epoch 1/20
332/332 [==============================] - 12s 31ms/step - loss: 0.3235 - accuracy: 0.8521 - val_loss: 0.1201 - val_accuracy: 0.9461
Epoch 2/20
332/332 [==============================] - 10s 29ms/step - loss: 0.1567 - accuracy: 0.9392 - val_loss: 0.1552 - val_accuracy: 0.9419
Epoch 3/20
332/332 [==============================] - 10s 30ms/step - loss: 0.1454 - accuracy: 0.9439 - val_loss: 0.1202 - val_accuracy: 0.9491
Epoch 4/20
332/332 [==============================] - 9s 28ms/step - loss: 0.1348 - accuracy: 0.9505 - val_loss: 0.1026 - val_accuracy: 0.9597
Epoch 5/20
332/332 [==============================] - 9s 28ms/step - loss: 0.1038 - accuracy: 0.9608 - val_loss: 0.1045 - val_accuracy: 0.9608
Epoch 6/20
332/332 [==============================] - 10s 30ms/step - loss: 0.1150 - accuracy: 0.9576 - val_loss: 0.0930 - val_accuracy: 0.9627
Epoch 7/20
332/332 [==============================] - 9s 28ms/step - loss: 0.0914 - accuracy: 0.9673 - val_loss: 0.1104 - val_accuracy: 0.

In [12]:
y_pred_binary = (model_w_binary.predict(X_test) > 0.5).astype(int)
y_true_binary = y_test 
# Calcular F1-score, precisión y recall
f1 = f1_score(y_true_binary, y_pred_binary)
precision = precision_score(y_true_binary, y_pred_binary)
recall = recall_score(y_true_binary, y_pred_binary)

# Imprimir los resultados
print("Resultados del modelo de clasificación binaria para datos de muñeca: ")
print(f'F1-score: {f1:.4f}')
print(f'Precisión: {precision:.4f}')
print(f'Recall: {recall:.4f}') # Recall es la Tasa de verdaderos positivos

# Imprimir la matriz de confusión
conf_matrix = confusion_matrix(y_true_binary, y_pred_binary)
print('Matriz de confusión:')
print(conf_matrix)

83/83 [==============================] - 1s 9ms/step
Resultados del modelo de clasificación binaria para datos de muñeca: 
F1-score: 0.9904
Precisión: 0.9950
Recall: 0.9858
Matriz de confusión:
[[ 813    9]
 [  26 1804]]


In [14]:
# Generar un modelo CNN para la clasificaión de tres estados para los datos de muñeca
# Modificar el modelo anterior para adaptar a la nueva función
X_segments_reshaped_2d = X_segments_reshaped.reshape(-1, num_features)

# Normalizar datos
scaler = StandardScaler()
X_segments_normalized_2d = scaler.fit_transform(X_segments_reshaped_2d)

# Redimensionar los datos normalizados a 3D
X_segments_normalized = X_segments_normalized_2d.reshape(X_segments_reshaped.shape)

# One hot encoding, num_classes = 4 porque los valores de label son de 1 a 3, tiene que ser mayor que 3 para que lo incluya
y_segments_one_hot = to_categorical(y_segments, num_classes=4) 

# Dividir datos para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_segments_normalized, y_segments_one_hot, test_size=0.2, random_state=42)

input_layer = Input(shape=(sequence_length, num_features))

def conv_block(x, filters, kernel_size, stride, pool_size, pool_stride):
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=stride, activation='relu')(x)
    x = MaxPooling1D(pool_size=pool_size, strides=pool_stride)(x)
    return x

output = conv_block(input_layer, filters=16, kernel_size=15, stride=2, pool_size=4, pool_stride=4)
output = conv_block(output, filters=32, kernel_size=10, stride=1, pool_size=2, pool_stride=2)

# Conexión completa entre layers
x = Dense(32, activation='relu')(Flatten()(output))
x = Dropout(0.5)(x)
x = Dense(16, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layer para 3 clases, donde 4 es el num_classes
output_layer = Dense(4, activation='softmax')(x)

# Generar el modelo para clasificación de 3 estados
model_w_three = Model(inputs=input_layer, outputs=output_layer)

model_w_three.summary()

optimizer = Adam(lr=0.01)
model_w_three.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model_w_three.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model_w_three.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación de tres estados para datos de muñeca: {accuracy * 100:.2f}%')


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 3500, 6)]         0         
                                                                 
 conv1d_3 (Conv1D)           (None, 1743, 16)          1456      
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 435, 16)           0         
 g1D)                                                            
                                                                 
 conv1d_4 (Conv1D)           (None, 426, 32)           5152      
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 213, 32)           0         
 g1D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 6816)              0   

In [15]:
y_pred_three = np.argmax(model_w_three.predict(X_test), axis=1)
y_true_three = np.argmax(y_test, axis=1)

f1 = f1_score(y_true_three, y_pred_three, average='weighted')
precision = precision_score(y_true_three, y_pred_three, average='weighted')
recall = recall_score(y_true_three, y_pred_three, average='weighted')

# Imprimir los reesultados
print("Resultados del modelo de clasificación de tres estados para datos de muñeca: ")
print(f'F1-score: {f1:.4f}')
print(f'Precisión: {precision:.4f}')
print(f'Recall: {recall:.4f}')

# Imprimir la matriz de confusión
conf_matrix = confusion_matrix(y_true_three, y_pred_three)
print('Matriz de confusión:')
print(conf_matrix)

83/83 [==============================] - 1s 8ms/step
Resultados del modelo de clasificación de tres estados para datos de muñeca: 
F1-score: 0.9494
Precisión: 0.9503
Recall: 0.9491
Matriz de confusión:
[[1273   27   49]
 [   8  798   16]
 [  27    8  446]]
